In [12]:
# Extract from Openweathermap API(live weather details)

import requests
import psycopg2
from datetime import datetime

API_KEY = "d6dbc559887d2c7a79a59caad00d9439"
BASE_URL = "http://api.openweathermap.org/data/2.5/weather"
city_name = "London" # mention the country name

# Fetch weather data from OpenWeatherMap API
params = {
    "q": city_name,
    "appid": API_KEY,
    "units": "metric" 
}
response = requests.get(BASE_URL, params=params)

if response.status_code == 200:
    data = response.json()
    # Extract relevant fields
    weather_data = {
        "city": data["name"],
        "datetime": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
        "temperature": data["main"]["temp"],
        "humidity": data["main"]["humidity"],
        "wind_speed": data["wind"]["speed"] * 3.6,  # Convert m/s to km/h
        "description": data["weather"][0]["description"]
    }
else:
    print("Failed to fetch data:", response.json())
    exit()

# Connect to PostgreSQL
conn = psycopg2.connect(
    dbname="weather",  
    user='postgres', 
    password="KARU55bime22", 
    host="localhost", 
    port="5432"  
)

cursor = conn.cursor()

# Insert weather data into the table
insert_query = """
INSERT INTO api_data (city, datetime, temperature, humidity, wind_speed, description)
VALUES (%s, %s, %s, %s, %s, %s)
"""
cursor.execute(insert_query, (
    weather_data["city"],
    weather_data["datetime"],
    weather_data["temperature"],
    weather_data["humidity"],
    weather_data["wind_speed"],
    weather_data["description"]
))
conn.commit()

# Query the database to check the contents
select_query = "SELECT * FROM api_data LIMIT 5;"
cursor.execute(select_query)
result = cursor.fetchall()

# Close the connection
cursor.close()
conn.close()

# Display the result
print("Inserted Weather Data:", result)


Inserted Weather Data: [('London', datetime.datetime(2024, 11, 28, 16, 12, 46), 3.11, 90, 1.836, 'overcast clouds')]


In [ ]:
# Extract the data from several contries

import requests
import psycopg2
from datetime import datetime
import time

# API key and base URL for OpenWeatherMap
API_KEY = "d6dbc559887d2c7a79a59caad00d9439"
BASE_URL = "http://api.openweathermap.org/data/2.5/weather"

# Sample list of cities and their countries
cities = [
    {"name": "New York", "country": "USA"},
    {"name": "London", "country": "UK"},
    {"name": "Tokyo", "country": "Japan"},
    {"name": "Delhi", "country": "India"},
    {"name": "Sydney", "country": "Australia"}
]

# Connect to database(PostgreSQL)
conn = psycopg2.connect(
    dbname="weather", 
    user="postgres", 
    password="KARU55bime22",
    host="localhost", 
    port="5432"  
)
cursor = conn.cursor()

# Loop through the city list and fetch weather data
for city in cities:
    params = {
        "q": city["name"],
        "appid": API_KEY,
        "units": "metric"
    }
    response = requests.get(BASE_URL, params=params)

    if response.status_code == 200:
        data = response.json()
        # Extract relevant fields
        weather_data = {
            "city": data["name"],
            "country": city["country"],
            "datetime": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
            "temperature": data["main"]["temp"],
            "humidity": data["main"]["humidity"],
            "wind_speed": data["wind"]["speed"] * 3.6,  # Convert m/s to km/h
            "description": data["weather"][0]["description"]
        }
        
        # Insert weather data into the table
        insert_query = """
        INSERT INTO api_data (city, country, datetime, temperature, humidity, wind_speed, description)
        VALUES (%s, %s, %s, %s, %s, %s, %s)
        """
        cursor.execute(insert_query, (
            weather_data["city"],
            weather_data["country"],
            weather_data["datetime"],
            weather_data["temperature"],
            weather_data["humidity"],
            weather_data["wind_speed"],
            weather_data["description"]
        ))
        conn.commit()
        print(f"Inserted data for {city['name']}")
    else:
        print(f"Failed to fetch data for {city['name']}: {response.status_code}")

    # Pause to avoid hitting the rate limit
    time.sleep(1)  # Adjust this based on your API plan

# Query the database to check the contents
select_query = "SELECT * FROM api_data LIMIT 10;"
cursor.execute(select_query)
result = cursor.fetchall()

# Close the connection
cursor.close()
conn.close()

# Display the result
print("Inserted Weather Data:", result)


Inserted data for New York
Inserted data for London
Inserted data for Tokyo
Inserted data for Delhi
Inserted data for Sydney
Inserted Weather Data: [('New York', 'USA', datetime.datetime(2024, 11, 27, 10, 16, 37), 6.85, 62, 14.472, 'few clouds'), ('London', 'UK', datetime.datetime(2024, 11, 27, 10, 16, 38), 8.15, 90, 22.212, 'light rain'), ('Tokyo', 'Japan', datetime.datetime(2024, 11, 27, 10, 16, 39), 19.34, 69, 9.251999999999999, 'few clouds'), ('Delhi', 'India', datetime.datetime(2024, 11, 27, 10, 16, 41), 20.05, 49, 5.5440000000000005, 'smoke'), ('Sydney', 'Australia', datetime.datetime(2024, 11, 27, 10, 16, 42), 35.69, 44, 23.471999999999998, 'overcast clouds'), ('New York', 'USA', datetime.datetime(2024, 11, 28, 14, 48, 27), 7.16, 65, 14.832, 'light rain'), ('London', 'UK', datetime.datetime(2024, 11, 28, 14, 48, 28), 1.28, 91, 3.708, 'overcast clouds'), ('Tokyo', 'Japan', datetime.datetime(2024, 11, 28, 14, 48, 29), 13.35, 32, 31.5, 'few clouds'), ('Delhi', 'India', datetime.dat

In [14]:
#Extract from Covid-19 API

import requests
import psycopg2
from datetime import datetime

# API URL for COVID-19 historical data
API_URL = "https://disease.sh/v3/covid-19/historical/all?lastdays=all"
response = requests.get(API_URL)

if response.status_code == 200:
    data = response.json()

    # Process the data (keys: 'cases', 'deaths', 'recovered')
    covid_data = []
    for date, cases in data["cases"].items():
        covid_data.append({
            "date": datetime.strptime(date, "%m/%d/%y").strftime("%Y-%m-%d"),
            "cases": cases,
            "deaths": data["deaths"][date],
            "recovered": data["recovered"][date],
        })
else:
    print("Failed to fetch data:", response.json())
    exit()

conn = psycopg2.connect(
    dbname="covid",
    user='postgres',
    password="KARU55bime22",
    host="localhost", 
    port="5432"
)

cursor = conn.cursor()

conn.commit()

# Insert COVID-19 data into the table
insert_query = """
INSERT INTO covid_data (date, cases, deaths, recovered)
VALUES (%s, %s, %s, %s)
ON CONFLICT (date) DO NOTHING;
"""

for record in covid_data:
    cursor.execute(insert_query, (
        record["date"],
        record["cases"],
        record["deaths"],
        record["recovered"]
    ))

conn.commit()

# Query the database to check the contents
select_query = "SELECT * FROM covid_data LIMIT 5;"
cursor.execute(select_query)
result = cursor.fetchall()

# Close the connection
cursor.close()
conn.close()

# Display the result
print("Inserted COVID-19 Data:", result)


Inserted COVID-19 Data: [(1, datetime.date(2020, 1, 22), 557, 17, 30), (2, datetime.date(2020, 1, 23), 657, 18, 32), (3, datetime.date(2020, 1, 24), 944, 26, 39), (4, datetime.date(2020, 1, 25), 1437, 42, 42), (5, datetime.date(2020, 1, 26), 2120, 56, 56)]
